In [1]:
import pandas as pd
import numpy as np

def display_full(x):
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 20000,
                           "display.max_colwidth", None,
                           ):
        print(x)

In [2]:
data = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_model_classification_training.xlsx")
display_full(data.head(2))

        patient_ID  scan_date                                                                                                SUV_MIP                                                                                                CT_MIP                                                                                                SUV_bone                                                                                                CT_bone                                                                                                SUV_lean                                                                                                CT_lean                                                                                                SUV_adipose                                                                                                CT_adipose                                                                                                SUV_air                                    

In [6]:
data["diagnosis_groups"].value_counts()

diagnosis_groups
C81    302
C83    223
C85     97
C82     51
C84     28
C88     11
C86      8
Name: count, dtype: int64

In [ ]:
metadata = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/Finalized_dataset_1805_exams_with_Age.xlsx")
print(metadata.columns)
metadata.head(2)

In [ ]:
metadata = metadata[["npr", "scan_date", "patient_age"]]
print(metadata.shape)
metadata.drop_duplicates(inplace=True)
print(metadata.shape)
metadata.head(2)

In [ ]:
final = pd.merge(data, metadata, how="inner", left_on=["patient_ID", "scan_date"], right_on=["npr", "scan_date"], suffixes=["_l","_r"])
final.head(2)

In [ ]:
final_data = final.drop(columns=["npr"])

In [ ]:
final_data.head()

In [ ]:
final_data.to_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_training_regression.xlsx", index=False)

##### EXTRA CODE

In [ ]:
testing = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_training_regression.xlsx")

In [ ]:
testing

In [ ]:
sort_test = testing.sort_values(by="patient_ID")
final_test = sort_test.drop(columns="Unnamed: 0").reset_index(drop=True)
final_test

In [ ]:
train_data = final_test[:int(final_test.shape[0] * 0.7)].patient_ID.tolist()
train = final_test[final_test.patient_ID.isin(train_data)]
val = final_test[~final_test.patient_ID.isin(train_data)]

In [ ]:
train.shape

In [ ]:
val.shape

In [ ]:
train["patient_age"].nunique()

In [ ]:
output_path = "/media/andres/T7 Shield1/UCAN_project/collages/"
df_for_collages = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/df_for_collages.xlsx")
df_for_collages.shape

In [ ]:
df_for_collages["patient_ID"].nunique()

In [ ]:
df_for_collages["scan_date"].nunique()

### Updating file names for new system

##### regression

In [ ]:
testing = pd.read_excel("/home/ashish/Ashish/UCAN/ReshapedCollages/dataset_for_model_training.xlsx")

In [ ]:
display_full(testing.head(2))

In [ ]:
testing = testing.replace('/media/andres/T7 Shield1/UCAN_project/collages', '/home/ashish/Ashish/UCAN/ReshapedCollages/collages/reshaped_collages', regex=True)
display_full(testing.head(2))

In [ ]:
testing.drop(columns='Unnamed: 0', inplace=True)
display_full(testing.head(2))

In [ ]:
testing.to_excel("/home/ashish/Ashish/UCAN/ReshapedCollages/dataset_for_model_training_v1.xlsx", index=False)

##### classification

In [ ]:
#classification_data = pd.read_excel("/home/ashish/Ashish/UCAN/dataset_for_training_classification.xlsx")
classification_data = pd.read_excel("/home/ashish/Ashish/UCAN/ReshapedCollages/dataset_for_model_training.xlsx")
classification_data.shape

In [ ]:
display_full(classification_data.head(2))

In [ ]:
classification_data = classification_data.replace('/media/andres/T7 Shield1/UCAN_project', '/home/ashish/Ashish/UCAN/ReshapedCollages', regex=True)
#classification_data.drop(columns=["patient_age"], inplace=True)
classification_data.patient_ID = classification_data.patient_ID.str.replace(" ","")
display_full(classification_data.head(2))

In [ ]:
classification_data.shape

In [ ]:
#clinicaldata_path = "/home/ashish/Ashish/UCAN/U-CAN Lymfom - Ahlström 211021 - utlämnat_pnr_removed.xlsx"
clinicaldata_path = "/home/ashish/Ashish/UCAN/20231129_UCAN_lym_export/Lymphoma_Octopus_Export 2023-11-29_C80_only_n1291_incl_YYYY_MM_pnr_removed.xlsx"
clinical_data = pd.read_excel(clinicaldata_path, header=0)
print(clinical_data.columns)
clinical_data = clinical_data.sort_values("personReference")
clinical_data = clinical_data[["personReference", "BirthYear", "BirthMonth", "consentDate", "translatedDiagnosis", "sex"]].copy()
clinical_data.columns = clinical_data.columns.str.strip()
clinical_data.npr = clinical_data.personReference.str.replace(" ","")
print(clinical_data.shape)
print(clinical_data.npr.nunique())
display_full(clinical_data.head(2))

In [ ]:
classification_data_v1 = pd.merge(classification_data, clinical_data, how="inner", left_on=["patient_ID"], right_on=["personReference"], suffixes=['','_'])
print(classification_data_v1.shape)
display_full(classification_data_v1.head(2))

In [ ]:
classification_data_v1.personReference.nunique()

In [ ]:
classification_data_v1["diagnosis_groups"] = classification_data_v1["translatedDiagnosis"].apply(lambda x: x[:3])
#["diaGroups_GT"] = np.where(classification_data_v1["diaGroups_new"]=="C83", 1, np.where(classification_data_v1["diaGroups_new"]=="C81", 2, 3))

In [ ]:
classification_data_v1.groupby("diagnosis_groups")['patient_ID'].nunique().sort_values(ascending=False)

In [ ]:
classification_data_v1["patientDOB"] = classification_data_v1.apply(lambda x: str(x["BirthYear"]) + (str(x["BirthMonth"]) if len(str(x["BirthMonth"]))>1 else "0" + str(x["BirthMonth"])) + "15", axis=1)
classification_data_v1[["scan_date", "patientDOB"]][:5]

In [ ]:
from datetime import date

classification_data_v1["scan_date"] = pd.to_datetime(classification_data_v1["scan_date"], format="%Y%m%d")
classification_data_v1["patientDOB"] = pd.to_datetime(classification_data_v1["patientDOB"], format="%Y%m%d")
classification_data_v1["consentDate"] = pd.to_datetime(classification_data_v1["consentDate"], format="%Y-%m-%d")
classification_data_v1[["patientDOB", "scan_date", "consentDate"]][:5]

In [ ]:
def calculateAge(dob, scan_date):
    return scan_date.year - dob.year - ((scan_date.month, scan_date.day) < (dob.month, dob.day))

In [ ]:
classification_data_v1["age"] = classification_data_v1.apply(lambda x: calculateAge(x["patientDOB"], x["scan_date"]), axis=1)
classification_data_v1[["patientDOB", "scan_date", "age"]][:5]

In [ ]:
classification_data_v1["diff_date (abs(sd-cd))"] = classification_data_v1.apply(lambda x: abs((x["scan_date"] - x["consentDate"])/np.timedelta64(1, 'D')), axis=1)

In [ ]:
display_full(classification_data_v1)

In [ ]:
classification_data_v1.columns

In [ ]:
classification_data_v2 = classification_data_v1[['patient_ID', 'patientDOB', 'scan_date', 'consentDate', 'diff_date (abs(sd-cd))', 'sex', 'patient_age', 'diagnosis_groups', 
                                                 'translatedDiagnosis', 'SUV_MIP', 'CT_MIP', 'SUV_bone', 'CT_bone', 'SUV_lean', 'CT_lean', 'SUV_adipose', 
                                                 'CT_adipose', 'SUV_air', 'CT_air']].copy()
classification_data_v2 = classification_data_v2.rename(columns={'patientDOB':'dob', 'patient_age':'age', 'consentDate':'consent_date', 'translatedDiagnosis':'diagnosis'})
classification_data_v2 = classification_data_v2.sort_values(["patient_ID", "scan_date"])
display_full(classification_data_v2.head(4))

In [ ]:
display_full(classification_data_v2)

In [ ]:
classification_data_v2.patient_ID.nunique()

In [ ]:
#classification_data_v2[classification_data_v2.age != classification_data_v2.patient_age]

In [ ]:
classification_data_v2.to_excel("/home/ashish/Ashish/UCAN/dataset_for_training_366patients_clinical20231129.xlsx", index=False)

### select baseline scans with closest date to consent date

In [ ]:
classification_data_v3 = classification_data_v2.copy()
display_full(classification_data_v3.head(3))

In [ ]:
classification_data_v3["rank"] = classification_data_v3.groupby("patient_ID")["diff_date (abs(sd-cd))"].rank(method="min", ascending=True)
display_full(classification_data_v3[['patient_ID', 'dob', 'scan_date', 'consent_date', 'diff_date (abs(sd-cd))', "rank"]].sort_values(["patient_ID","rank"]))

In [ ]:
df1 = classification_data_v3[classification_data_v3["rank"]==1].copy()
df1.drop(columns="rank", inplace=True)
df1.shape

In [ ]:
nan_df = classification_data_v3[classification_data_v3["rank"].isin([np.nan])].copy()
nan_df["rank1"] = nan_df.groupby("patient_ID")["scan_date"].rank(method="first", ascending=False)
nan_df[["patient_ID", "scan_date", "rank1"]]

In [ ]:
df2 = nan_df[nan_df["rank1"]==1].copy()
df2.drop(columns=["rank", "rank1"], inplace=True)
df2.head(2)

In [ ]:
classification_data_v4 = pd.concat([df1,df2], ignore_index=True).reset_index(drop=True)
print(classification_data_v4.shape)
display_full(classification_data_v4.head())

In [ ]:
classification_data_v4.to_excel("/home/ashish/Ashish/UCAN/dataset_for_training_366patients_baseline_scans_clinical20231129.xlsx", index=False)

### remove nan

In [ ]:
images_with_nan_values = classification_data_v1.copy()
images_with_nan_values['incorrect_projection'] = 'False'

# Loop through each row of the dataset and check if there are any arrays with NaN values in them
for index, row in images_with_nan_values.iterrows():
    arr = np.load(row["SUV_MIP"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_MIP"])
        continue

    arr = np.load(row["SUV_bone"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_bone"])
        continue
        
    arr = np.load(row["SUV_lean"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_lean"])
        continue
        
    arr = np.load(row["SUV_adipose"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_adipose"])
        continue
        
    arr = np.load(row["SUV_air"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_air"])
        continue
        
    arr = np.load(row["CT_MIP"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_MIP"])
        continue
        
    arr = np.load(row["CT_bone"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_bone"])
        continue
        
    arr = np.load(row["CT_lean"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_lean"])
        continue
        
    arr = np.load(row["CT_adipose"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_adipose"])
        continue
        
    arr = np.load(row["CT_air"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_air"])
        continue

display_full(images_with_nan_values[images_with_nan_values['incorrect_projection'] == 'True'])

In [ ]:
classification_data_v2 = images_with_nan_values[images_with_nan_values['incorrect_projection'] == 'False'].copy()
classification_data_v2 = classification_data_v2[classification_data_v2.sex.isin(["MALE", "FEMALE"])].copy()

classification_data_v2['sex'] = np.where(classification_data_v2['sex']=="MALE", 1, 0)

classification_data_v2.to_excel("/home/ashish/Ashish/UCAN/dataset_for_training_classification_v2.xlsx", index=False)

In [ ]:
classification_data_v2.npr.nunique()

In [ ]:
classification_data_v2 = classification_data_v1[classification_data_v1.sex.isin(["MALE", "FEMALE"])].copy()

classification_data_v2['sex'] = np.where(classification_data_v2['sex']=="MALE", 1, 0)
display_full(classification_data_v2.head(2))
classification_data_v2.to_excel("/home/ashish/Ashish/UCAN/ReshapedCollages/dataset_for_training_classification_v2.xlsx", index=False)

In [ ]:
classification_data_v2.sex.value_counts()
classification_data_v2

In [ ]:
df= pd.read_excel("/home/ashish/Ashish/UCAN/ReshapedCollages/dataset_for_training_classification_v2.xlsx")
df.columns

In [ ]:
df["D_group"] = df["Diagnosis"].apply(lambda x: x[:3])
df.head(3)

In [ ]:
df.groupby("D_group")['npr'].nunique().sort_values(ascending=False)

In [ ]:
86+25+5+4+3+3+2+1+1

In [ ]:
display_full(df.groupby("Diagnosis")['npr'].nunique().sort_values(ascending=False))

In [ ]:
display_full(df.Diagnosis.value_counts())

In [2]:
df = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_model_classification_training.xlsx")

In [3]:
df = df["patient_ID"]

In [4]:
df.nunique()

364

In [5]:
df.shape

(720,)

: 